In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [2]:
f2 = open('task2_lemmas_train_single', 'w')
with open('task2_lemmas_train') as f:
    for line in f:
        f2.write(str(line.split(',')[0]) + ',' + str(line.split(',')[1]) + ',' + str(line.split(',')[2]) + '\n')
f.close()
f2.close()

In [3]:
data = pd.read_csv('task2_lemmas_train_single', sep=',')

In [4]:
train = data[:len(data)/10*9]
test = data[len(data)/10*9:]
resultative = pd.read_csv('task2_lemmas_test', sep=',')
example = pd.read_csv('task2_lemmas_test', sep=',')

In [5]:
#найти наибольшую общую последовательность
def lcs(a, b):
    a, b = (a, b) if len(a) < len(b) else (b, a)
    pos = 0
    substr = ""
    search_str = a[:]
    while ( pos != len(a) ):
        while ( len(search_str) > len(substr) ):
            if search_str in b:
                substr = search_str[:]
                break
            else:
                search_str = search_str[:-1]
        pos += 1
        search_str = a[pos:]
    return substr

In [6]:
lcs('vergognerete', 'vergognare+V')

'vergogn'

In [7]:
train.iat[0,1]

'vergognerete'

In [8]:
train['root'] = [lcs(train.iat[i,1],train.iat[i,2]) for i in range(len(train))]
train['y2'] = [train.iat[i,2][:-2] for i in range(len(train))]
train['len_suf'] = [len(train.iat[i,1]) - len(train.iat[i,3]) for i in range(len(train))]
train['part'] = [train.iat[i,2][-1] for i in range(len(train))]

/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [9]:
train['suf_y'] = [train.iat[i,4][-int(train.iat[i,5]):] for i in range(len(train))]

/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
test['root'] = [lcs(test.iat[i,1],test.iat[i,2]) for i in range(len(test))]
test['y2'] = [test.iat[i,2][:-2] for i in range(len(test))]
test['len_suf'] = [len(test.iat[i,1]) - len(test.iat[i,3]) for i in range(len(test))]
test['part'] = [test.iat[i,2][-1] for i in range(len(test))]
test['suf_y'] = [test.iat[i,4][-int(test.iat[i,5]):] for i in range(len(test))]

/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [11]:
train[0:5]

,Id,X,y,root,y2,len_suf,part,suf_y
0,1,vergognerete,vergognare+V,vergogn,vergognare,5,V,gnare
1,2,amnistiavate,amnistiare+V,amnistia,amnistiare,4,V,iare
2,3,menomazione,menomazione+N,menomazione,menomazione,0,N,menomazione
3,4,sfaldavamo,sfaldare+V,sfalda,sfaldare,4,V,dare
4,5,sfodererei,sfoderare+V,sfoder,sfoderare,4,V,rare


In [12]:
test[0:5]

,Id,X,y,root,y2,len_suf,part,suf_y
106776,106777,disinteressato,disinteressato+A,disinteressato,disinteressato,0,A,disinteressato
106777,106778,corrispondemmo,corrispondere+V,corrisponde,corrispondere,3,V,ere
106778,106779,premunirebbero,premunire+V,premunire,premunire,5,V,unire
106779,106780,dedichiate,dedicare+V,dedic,dedicare,5,V,icare
106780,106781,cedoli,cedolare+V,cedol,cedolare,1,V,e


In [ ]:
#1. сколько букв надо отрезать
pipeline1 = Pipeline([("vectorizer", CountVectorizer(ngram_range=(1, 8), analyzer='char_wb')), ("algo", LogisticRegression())])
pipeline1.fit(train.X, train.len_suf)
preds1 = pipeline1.predict(test.X)
accuracy_score(test.len_suf, preds1)

0.93543492919757254

In [ ]:
#2. какой суффикс у начальной формы
pipeline2 = Pipeline([("vectorizer", CountVectorizer(ngram_range=(1, 8), analyzer='char_wb')), ("algo", LogisticRegression())])
pipeline2.fit(train.X, train.suf_y)
preds2 = pipeline2.predict(test.X)
print accuracy_score(test.suf_y, preds2)

In [ ]:
#3. какая часть речи
pipeline3 = Pipeline([("vectorizer", CountVectorizer(ngram_range=(1, 8), analyzer='char_wb')), ("algo", LogisticRegression())])
pipeline3.fit(train.X, train.part)
preds3 = pipeline3.predict(test.X)
print accuracy_score(test.part, preds3)

In [ ]:
preds = [str(test.iat(i,1)[:-int(preds1[i])]) + str(preds2[i]) + '+' +str(preds3[i]) for i in range(len(test))]
print accuracy_score(test.y, preds)

In [ ]:


'''result = pd.DataFrame({"Answer":preds, "Id":example.index})
result.set_index('Id', inplace=True)
result.to_csv('result5.csv')'''